APIキーの取得は[ここ](https://www.single-life.tokyo/google-maps-platform%ef%bc%88%e3%82%b0%e3%83%bc%e3%82%b0%e3%83%ab%e3%83%9e%e3%83%83%e3%83%97%ef%bc%89%e3%81%ae%e5%b0%8e%e5%85%a5%e6%96%b9%e6%b3%95%ef%bc%88%e6%ba%96%e5%82%99%e7%b7%a8%ef%bc%89/)で参照

Google Maps Elevation APIの利用料金は、通常、リクエストの数に基づいて計算されます。無料枠内では無料で利用できますが、無料枠を超えた場合は、超過したリクエストの数に応じて料金が発生します。

Google Maps Platformは、すべてのAPIに適用される月額200米ドルの無料利用枠を提供しています。これにはElevation APIも含まれます。Googleの価格表によると、Elevation APIのリクエストが無料枠を超えた後の料金は、1000リクエストごとに約0.5米ドルです。つまり、Elevation APIのみを利用する場合、200米ドルの無料利用枠で約400,000回の無料リクエストをサポートできることになります。

In [1]:
# API キー（自分のもの）
api_key = '##############'

In [2]:
import os
import re
import sys
from pathlib import Path
from os import path
import json
import time
import pyodbc
import webbrowser
import shutil
import csv
import pandas as pd
import datetime
import sklearn
from sklearn.neighbors import KNeighborsClassifier
import pyodbc
import numpy as np
import math
import datetime
import warnings
import requests
warnings.simplefilter('ignore')

In [3]:
#データベース接続
driver='{SQL Server}'
server = # server name
database = # db name
uid = # user id
pwd = # password
trusted_connection ='yes'

# AltitudeAllTableName = 
# AltitudeTableName = 
# EfficiencyTableName = 
# EfficiencyMaxTableName = 
# Cars_Tablename = 
# Place_Tablename = 

minTorque = 76  #EfficiencyMaxTableNameテーブルのTorqueカラムの最小値
maxTorque = 280 #EfficiencyMaxTableNameテーブルのTorqueカラムの最大値
maxRev = 9990   #EfficiencyMaxTableNameテーブルのRevカラムの最大値


#環境データ
myu = 0.015
rho = 1.22
GravityResistanceCoefficient = 9.80665
windSpeed = 0.0

# ヒュベニの公式
# https://butter-tiger.hatenablog.com/entry/2020/08/20/222650
POLE_RADIUS = 6356752  # 極半径(短半径)
EQUATOR_RADIUS = 6378137  # 赤道半径(長半径)
E = 0.081819191042815790  # 離心率
E2 = 0.006694380022900788  # 離心率の２乗

#車両データ
'''
Weight = 1600
FrontProjectedArea = 2.19
CdValue = 0.28
TireRadius = 0.3155
ReductionRatio = 7.9377
'''
'''
Weight = 1760
FrontProjectedArea = 2.24
CdValue = 0.28
TireRadius = 0.3234
ReductionRatio = 8.1938
'''
InverterEfficiency = 0.95
MaxDrivingPower = -30
#MaxDrivingForce = -0.15 * GravityResistanceCoefficient * Weight
CarOtherWeight = 80

In [4]:
def distance(_lat1, _long1, _lat2, _long2):
    lat1 = math.radians(_lat1)
    long1 = math.radians(_long1)
    lat2 = math.radians(_lat2)
    long2 = math.radians(_long2)
    m_lat = (lat1 + lat2) / 2  # 平均緯度
    d_lat = abs(lat1 - lat2)  # 緯度差
    d_lon = abs(long1 - long2)  # 経度差
    W = math.sqrt(1 - E2 * math.pow(math.sin(m_lat), 2))
    M = EQUATOR_RADIUS * (1 - E2) / math.pow(W, 3)  # 子午線曲率半径
    N = EQUATOR_RADIUS / W  # 卯酉線曲率半径
    # d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2) + math.pow(point_a.altitude-point_b.altitude,2))
    d = math.sqrt(math.pow(M * d_lat, 2) + math.pow(N * d_lon * math.cos(m_lat), 2))
    return d

In [5]:
# KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返すメソッド
def readLinksKML(filename):
    lineSirungTab1 = r"<LineString>"
    lineSirungTab2 = r"</LineString>"
    isLineString = 0
    f = open(filename, "r", encoding="UTF-8")
    datalist = f.readlines()
    GPSlist = [["Latitude", "Longitude"]]
    for data in datalist:
        if isLineString == 1:
            if lineSirungTab2 in data:
                # print('ここまでが対象データ')
                isLineString = 0
            else:
                target_data = data.replace(" ", "").replace(r",0", "")
                if re.match(r"[0-9]+.?[0-9]*,[0-9]+.?[0-9]*", target_data):
                    target_data2 = re.split(",", target_data)
                    appendList = [float(target_data2[1]), float(target_data2[0])]
                    GPSlist.append(appendList)
                    # print(appendList)
        elif isLineString == 0:
            if lineSirungTab1 in data:
                # print('ここからが対象データ')
                isLineString = 1
        else:
            print("よきせぬれーがい")
    GPSlist.remove(["Latitude", "Longitude"])
    return GPSlist


def calcDistance(list_GPS):
    list_lat_long_dist = [
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    ]
    before_lat = 0
    before_long = 0
    dist_sum = 0
    for row in list_GPS:
        if before_lat > 0:
            dist = distance(before_lat, before_long, row[0], row[1])
            appendList = [before_lat, before_long, row[0], row[1], dist]
            list_lat_long_dist.append(appendList)
            dist_sum = dist_sum + dist
            # print(appendList)
        before_lat = row[0]
        before_long = row[1]
    list_lat_long_dist.remove(
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    )
    return list_lat_long_dist, dist_sum

def normalizedCoordinatesGenerator(list_lat_long_dist, normalized_dist):
    normalizedList = [["Latitude", "Longitude"]]
    rest = normalized_dist
    _lat = list_lat_long_dist[0][0]
    _long = list_lat_long_dist[0][1]
    appendList = [_lat, _long]
    normalizedList.append(appendList)
    coordinatesNumber = 1

    for row in list_lat_long_dist:
        row_loop = 0
        if rest > row[4]:
            rest = rest - row[4]
            # print('rest:', str(rest))
        else:
            while rest + normalized_dist * row_loop < row[4]:
                _lat = (row[2] - row[0]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[0]
                _long = (row[3] - row[1]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[1]
                appendList = [_lat, _long]
                normalizedList.append(appendList)
                coordinatesNumber = coordinatesNumber + 1
                row_loop = row_loop + 1
                # print('rest:', str(rest))
            rest = rest - row[4] + normalized_dist * row_loop
    
    return normalizedList, coordinatesNumber

def GPSListGenerator_segment(filename,speed):
    list_GPS = readLinksKML(filename)  # KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返す
    # print(list_GPS)    #(lat, long)
    list_lat_long_dist, dist_sum = calcDistance(list_GPS)
    NORMALIZED_DISTANCE = speed

    normalized_dist = NORMALIZED_DISTANCE

    list_normalized_lat_long, coordinatesNumber = normalizedCoordinatesGenerator(
        list_lat_long_dist, normalized_dist
    )
    
    return list_normalized_lat_long

In [6]:
now_jst = datetime.datetime.now()
now_jst_result = str(now_jst.strftime('%Y%m%d-%H%M%S'))
#FOLDER_PATH = "\\{}".format(now_jst_result)  # 作業フォルダ
FOLDER_PATH = ""  # 作業フォルダ
KML_PATH = rf"\KML_DrivingRoute"  # 入力フォルダ
KML_inserted_PATH = rf"\KML_DrivingRoute_inserted"  # 入力済みフォルダ
CSV_PATH = rf"\CSV_elevation"  # 出力フォルダ
#ACCESS_NUMBER = 5  # いつもNAVI APIアクセス数（1アクセスにつき100入力まで）
#CONSTANT_SPEED = 100 # 定速速度はファイル名から自動で判別するようにする（この値はデフォルト値）

# ヒュベニの公式
# https://butter-tiger.hatenablog.com/entry/2020/08/20/222650
POLE_RADIUS = 6356752  # 極半径(短半径)
EQUATOR_RADIUS = 6378137  # 赤道半径(長半径)
E = 0.081819191042815790  # 離心率
E2 = 0.006694380022900788  # 離心率の２乗

def distance(_lat1, _long1, _lat2, _long2):
    lat1 = math.radians(_lat1)
    long1 = math.radians(_long1)
    lat2 = math.radians(_lat2)
    long2 = math.radians(_long2)
    m_lat = (lat1 + lat2) / 2  # 平均緯度
    d_lat = abs(lat1 - lat2)  # 緯度差
    d_lon = abs(long1 - long2)  # 経度差
    W = math.sqrt(1 - E2 * math.pow(math.sin(m_lat), 2))
    M = EQUATOR_RADIUS * (1 - E2) / math.pow(W, 3)  # 子午線曲率半径
    N = EQUATOR_RADIUS / W  # 卯酉線曲率半径
    # d = math.sqrt(math.pow(M*d_lat,2) + math.pow(N*d_lon*math.cos(m_lat),2) + math.pow(point_a.altitude-point_b.altitude,2))
    d = math.sqrt(math.pow(M * d_lat, 2) + math.pow(N * d_lon * math.cos(m_lat), 2))
    return d


def search_filelist():
    
    p = Path(rf"{os.getcwd()}{KML_PATH}")
    print(p)

    filenames = []

    for file in p.iterdir():
        if file.is_dir():
            continue

        # pathstrings = str(file).split('/')
        # filename = file.name
        # print(pathstrings)
        if re.match(".+" + r".kml", file.name):
            print(file.name, ":KML file")
            filenames.append(
                rf"{os.getcwd()}{KML_PATH}\{file.name}"
            )
        else:
            print(file.name, ":not KML file:")
        # filenames.append(file.name)
    return filenames


# KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返すメソッド
def readLinksKML(filename):
    lineSirungTab1 = r"<LineString>"
    lineSirungTab2 = r"</LineString>"
    isLineString = 0
    f = open(filename, "r", encoding="UTF-8")
    datalist = f.readlines()
    GPSlist = [["Latitude", "Longitude"]]
    for data in datalist:
        if isLineString == 1:
            if lineSirungTab2 in data:
                # print('ここまでが対象データ')
                isLineString = 0
            else:
                target_data = data.replace(" ", "").replace(r",0", "")
                if re.match(r"[0-9]+.?[0-9]*,[0-9]+.?[0-9]*", target_data):
                    target_data2 = re.split(",", target_data)
                    appendList = [float(target_data2[1]), float(target_data2[0])]
                    GPSlist.append(appendList)
                    # print(appendList)
        elif isLineString == 0:
            if lineSirungTab1 in data:
                # print('ここからが対象データ')
                isLineString = 1
        else:
            print("よきせぬれーがい")
    GPSlist.remove(["Latitude", "Longitude"])
    return GPSlist


def calcDistance(list_GPS):
    list_lat_long_dist = [
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    ]
    before_lat = 0
    before_long = 0
    dist_sum = 0
    for row in list_GPS:
        if before_lat > 0:
            dist = distance(before_lat, before_long, row[0], row[1])
            appendList = [before_lat, before_long, row[0], row[1], dist]
            list_lat_long_dist.append(appendList)
            dist_sum = dist_sum + dist
            # print(appendList)
        before_lat = row[0]
        before_long = row[1]
    list_lat_long_dist.remove(
        ["Latitude1", "Longitude1", "Latitude2", "Longitude2", "Distance"]
    )
    return list_lat_long_dist, dist_sum


def normalizedCoordinatesGenerator(list_lat_long_dist, normalized_dist):
    normalizedList = [["Latitude", "Longitude"]]
    rest = normalized_dist
    _lat = list_lat_long_dist[0][0]
    _long = list_lat_long_dist[0][1]
    appendList = [_lat, _long]
    normalizedList.append(appendList)
    coordinatesNumber = 1

    for row in list_lat_long_dist:
        row_loop = 0
        if rest > row[4]:
            rest = rest - row[4]
            # print('rest:', str(rest))
        else:
            while rest + normalized_dist * row_loop < row[4]:
                _lat = (row[2] - row[0]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[0]
                _long = (row[3] - row[1]) * (rest + normalized_dist * row_loop) / row[
                    4
                ] + row[1]
                appendList = [_lat, _long]
                normalizedList.append(appendList)
                coordinatesNumber = coordinatesNumber + 1
                row_loop = row_loop + 1
                # print('rest:', str(rest))
            rest = rest - row[4] + normalized_dist * row_loop
    
    return normalizedList, coordinatesNumber

coordinatesNumber = 0.00
# MAIN STREAM
def GPSListGenerator(filename):
#     print("try generate {}.".format(filename))
    list_GPS = readLinksKML(filename)  # KMLファイルの中から，必要な座標列のみを抽出し，リスト形式で返す
    # print(list_GPS)    #(lat, long)
    list_lat_long_dist, dist_sum = calcDistance(list_GPS)
    # print(list_lat_long_dist)  #(lat1, long1, lat2, long2, dist)

    NORMALIZED_DISTANCE = 10000
    #print(__filename[-1])
    CONSTANCE_SPEED = 20
#     print(filename, "  speed:", CONSTANCE_SPEED)
    NORMALIZED_DISTANCE = CONSTANCE_SPEED * 1000 / 3600

    normalized_dist = NORMALIZED_DISTANCE
#     print(
#         "distance:",
#         str(round((dist_sum / 1000), 3)),
#         "km,  normalized_dist:",
#         str(normalized_dist),
#         "m",
#     )
    list_normalized_lat_long, coordinatesNumber = normalizedCoordinatesGenerator(
        list_lat_long_dist, normalized_dist
    )
#     print('now printing')
#     print(list_normalized_lat_long)
    return list_normalized_lat_long

#上のGPSListGeneratorのoutputをcsvファイルとして出力
def output_lat_long(filename, list_normalized_lat_long):
    filename_csv = filename.replace(KML_PATH, CSV_PATH).replace(r'.kml', r'.csv')
    print("output file:", filename_csv, "    座標数:", str(coordinatesNumber), "\n")
    # 2次元配列→CSV変換:https://rikei-danshi.work/entry/python-2darray-csv
    with open(filename_csv, "w") as file:
        writer = csv.writer(file, lineterminator="\n")
        writer.writerows(list_normalized_lat_long)


def move_file(filename):
    filename_kml_inserted = filename.replace(KML_PATH, KML_inserted_PATH)
    shutil.move(filename, filename_kml_inserted)


def display_folium_map_from_csv(file_path):
    output_folium_file_path = rf"{path.dirname(__file__)}{FOLDER_PATH}{CSV_FOLIUM_PATH}\{os.path.basename(file_path).split('.')[0]}.html"
    df_output_points = pd.read_csv(
        rf"{path.dirname(__file__)}{FOLDER_PATH}{CSV_PATH}\{os.path.basename(file_path).split('.')[0]}.csv"
    )
    folium_map = folium.Map(
        location=[
            df_output_points.at[len(df_output_points) // 2, "Latitude"],
            df_output_points.at[len(df_output_points) // 2, "Longitude"],
        ],
        zoom_start=13,
    )
    for index, point in df_output_points.iterrows():
        folium.Marker(location=[point["Latitude"], point["Longitude"]]).add_to(
            folium_map
        )
    folium_map.save(output_folium_file_path)
    webbrowser.open(output_folium_file_path, new=2)

In [7]:
 def generate_VCVR(kml_filename):
    Speed = 1
    distance_between_segments = Speed
    segment_distance = Speed
    #read kml and convert it to a DataFrame
    #仮想走行ログの座標群を生成する
    df_Coordinate = pd.DataFrame(GPSListGenerator_segment(kml_filename,Speed)[1:],columns =GPSListGenerator_segment(kml_filename,Speed)[0])
    Coordinate_id_series = list(range(len(df_Coordinate)))

    df_Coordinate['Coordinate_id'] = Coordinate_id_series
    return df_Coordinate

In [8]:
def get_elevation(df_GPS_):
    df_ = df_GPS_.copy()
    elevation_list = []
    
    for i in range(len(df_)):
        latitude = df_['Latitude'].iloc[i]
        longitude = df_['Longitude'].iloc[i]
    
        # API リクエストURL　構築
        url = f"https://maps.googleapis.com/maps/api/elevation/json?locations={latitude},{longitude}&key={api_key}"
        # リクエスト
        response = requests.get(url)
        # JSON変換
        data = response.json()
        # 標高取得
        elevation = int(float(data['results'][0]['elevation'])*1000)
        elevation_list.append(elevation)
#         print(i,len(df_),elevation)
        
    df_['ZENRIN_Elevation'] = elevation_list
    
        
    return df_

In [9]:
generate_VCVR(search_filelist()[0])

C:\Users\hexin\Desktop\google_api_ele\KML_DrivingRoute
8888.kml :KML file


,Latitude,Longitude,Coordinate_id
0,42.648400,141.631370,0
1,42.648404,141.631381,1
2,42.648407,141.631392,2
3,42.648411,141.631403,3
4,42.648415,141.631414,4
...,...,...,...
72422,43.175888,141.768178,72422
72423,43.175892,141.768188,72423
72424,43.175897,141.768199,72424
72425,43.175901,141.768209,72425


In [10]:
if __name__ == "__main__":
    for filename in search_filelist():
        print('--------------------------------')
        print("now: "+filename)
        #read kml and convert it to a DataFrame
        SemantickLink = search_filelist()[0].split('\\')[-1][0:3]
        df_GPS = generate_VCVR(filename)
        
        
        #ここでgoogle earth取得関数をいれる
        df_GPS_with_elevation = get_elevation(df_GPS)
        df_GPS_with_elevation['SemanticLink']  = SemantickLink
        df_GPS_with_elevation['adas_null']  = 'Google_earth_API'
        df_GPS_with_elevation = df_GPS_with_elevation[['SemanticLink','Latitude','Longitude','ZENRIN_Elevation','adas_null']]
        
        print('--------------------------------')
        print('取得完了')
        #save result to folder
        df_GPS_with_elevation.to_csv('CSV_elevation\\{}.csv'.format(SemantickLink))
        
        #display_folium_map_from_csv(filename)
        move_file(filename)

C:\Users\hexin\Desktop\google_api_ele\KML_DrivingRoute
8888.kml :KML file
--------------------------------
now: C:\Users\hexin\Desktop\google_api_ele\KML_DrivingRoute\8888.kml
C:\Users\hexin\Desktop\google_api_ele\KML_DrivingRoute
8888.kml :KML file
--------------------------------
取得完了
